# Brainhack Global 2021 - Intro to writing Python tools

## Pre-requisites
Before you run through this notebook, make sure you've completed the following steps:
- Installed conda or miniconda
- Created a conda environment from the "environment.yaml" file in the repository. To do this, run the following command in your terminal: `conda create -n simplesig python=3.8 numpy ipykernel`
- Made sure that the kernel for this notebook is the python installed in your new environment ("simplesig")

## Overview
Welcome to the Brainhack Global Australasia 2021 Python workshop! Over the course of this workshop, you'll learn how to create functional python-based analysis tools. We'll run through the following steps:
- Write python code to:
    - Generate a simple artificial signal
    - Run some simple frequency decomposition analyses on this signal
    - Plot the results with [holoviews](https://holoviews.org/getting_started/index.html)
- Implement your newly written code in functions
- Compile these functions into a pip-installable module
- Compile these functions into a command-line script

## Step 1: Import modules
Over the course of this tutorial, you'll have to load in a number of python "modules" to run the code. These are scripts that other people have written and made available for us to use, so that we don't have to write everything from scratch! By creating the conda environment at the begining of the session, we downloaded these scripts and are now able to use them in this script as long as we "import" the module that they are stored under. 

It's good practice to keep all of these imports in the same place, so as you need to add each new module in the tutorial, make sure you add the call to import them in the cell below. 

We'll start with just the [numpy](https://numpy.org/) package (_"The fundamental package for scientific computing with Python"_ ).

In [ ]:
# Import modules
import numpy as np

## Step 2: Learn about scipy.signal.gausspulse
When setting up a new data analysis pipeline, a good starting point is to use some artificial data in which you know the ground truth properties of the signal. In this tutorial we'll be running frequency decomposition analyses. We'll therefore begin by generating an artificial signal with a number of different frequency-specific signals embedded at different times without the signal. By doing this, we can evaluate how well different algorithms are able to extract this information from our signal. 

To generate these frequency-specific signals, we'll be using the [gausspulse](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.gausspulse.html) function from the scipy-signal toolbox. Lets begin by creating an example gaussian pulse so you can get an idea of how this function works. But first:

_Add `from scipy import signal` to the import modules cell above, and run it again._

In [ ]:
# Set parameters
frequency = 10 # get a signal at 10 Hz
width = 2 # What "width" in time should the gaussian envelope around our sin wave be
amplitude = 3 # What amplitude should the sinusoidal signal be?
fs = 1024 # set an artificial "sampling frequency"; the number of datapoints per second to generate
t = np.arange(-1, 1, 1/fs) # A vector of times that will correspond to our datapoints

# Generate the signal
pulse = signal.gausspulse(t, fc=frequency, bw=2/(frequency * width ))*amplitude

## Step 3: Visualise the gaussian pulse using Holoviews
In this tutorial, we'll use [holoviews](https://holoviews.org/getting_started/index.html) to plot our data. This is a powerful package for data visualisation that will allow us to create interactive graphs. Feel free to have a look at their website for some examples of the types of plots you can make. 

We'll use the [bokeh](https://bokeh.org/) extension of holoviews so that we can make our plots interactive. Bokeh also allows you to turn your plots into a widget which you can embed in a PDF or website, so that your readers can dynamically plot different aspects of your data (i.e. scroll through different EEG channels). 

To use this package, we'll have to add a couple more lines to our "import modules" cell above:
```
import holoviews as hv # This imports the holoviews module
hv.extension('bokeh') # Let's use interactive plots based on the 'bokeh' library
hv.renderer('bokeh').theme = 'dark_minimal' # We'll use a dark theme for our plots, because dark themes are cool. 
```

_Remember to run the cell again once you've added this code!_


In [ ]:
# Create a line plot with our signal
curve = hv.Curve((t, pulse), 'Time (s)', 'Signal Amplitude')

# Set some options for how our signal will be plotted and display the plot
curve.opts(hv.opts.Curve(line_width=1, line_color='yellow', width=500))

You should notice that the gaussian pulse above is centred around time: 0. To shift the pulse around in time, we can manipulate the `t` variable when generating the pulse; setting the time we would like the pulse to be centred around as time:0 for the signal generation. This shift is implemented this in the cell below. Play around with the shift and other signal properties until you feel like you understand what all the variables do. 

In [ ]:
# Set parameters
frequency = 10 # get a signal at 10 Hz
width = 2 # What "width" in time should the gaussian envelope around our sin wave be
amplitude = 3 # What amplitude should the sinusoidal signal be?
time = 5 # What time in the signal should the pulse be centred at?
fs = 1000 # set an artificial "sampling frequency"; the number of datapoints per second to generate
t = np.arange(0, 10, 1/fs) # A vector of times that will correspond to our datapoints

# Generate the signal
pulse = signal.gausspulse(t - time, fc=frequency, bw=2/(frequency * width)) * amplitude

# Create a line plot with our signal
curve = hv.Curve((t, pulse), 'Time (s)', 'Signal Amplitude')

# Set some options for how our signal will be plotted and display the plot
curve.opts(hv.opts.Curve(line_width=1, line_color='yellow', width=500))

## Step 4: List comprehension in Python - An elegent weapon for a more civilised script

Consider a situation where you want to perform the same operation on multiple sets of inputs, say from a list, and store the outputs in a second list. Where possible, the most efficient solution is to use a _vectorised_ function in which the operation is performed across all elements at once:

In [ ]:
# Generate a python list
angles = [10, 45, 80]

# convert our angles from degrees to radians, and then take the sin
x = np.sin(np.deg2rad(angles))

# Display the result
print(x)

There are, however, many situations in which vectorised functions are not available. Let's say you've created a complex function (but you haven't got the time or resources to vectorise it - in this case we rely on the built-in python math functions instead of the numpy vectorised functions):

In [ ]:
from math import sin, radians

def opposite(angle: float, hypotenuse: float) -> float:
    '''Calculates the length of a triangle's side opposite the angle.
    '''
    return sin(radians(angle)) * hypotenuse

If we try to call this function with our list of angles, we'll get an error. One option would be to use a for loop.

In [ ]:
# for each individual angle in the list of angles, call the function
# 'opposite' and append the return value to a list
x = []
hypotenuse = 10
for angle in angles:
    x.append(opposite(angle, hypotenuse))

# Display the result
print(x)

Here we had to initialise an empty list, then explicitly append to that list as we iterate through the angles. Because the goal in this case is simply to create a list, Python provides a more concise approach called a _list comprehension_:

In [ ]:
# iterate through values of orientation using list comprehension
x = [opposite(angle, hypotenuse) for angle in angles]

# Display the result
print(x)

It is even possible to iterate through multiple input lists simultaneously, by using a function called _zip_. This turns multiple iterable objects like lists, and turns them into an iterable list of _tuples_:

In [ ]:
# Generate an additional python list
hypotenuses = [3, 5, 11]

# iterate through values of orientation and hypotenuse using list comprehension
opposite = [opposite(angle, hypotenuse)
            for angle, hypotenuse in zip(angles, hypotenuses)]

# Display the result
print(opposite)

List comprehension can also include logic, and can even replace nested for loops! We won't cover these topics in this tutorial but feel free to look into what lists can do for you later. 

## Step 5: We're ready to make some artificial data!
As discussed at the beginning of the tutorial, we aim to make a simple artificial signal with some frequency specific signals embedded at specific times. To begin, lets set the parameters of this signal, we can always change these later to change different aspects of our signal. 

In [ ]:
# set signal parameters
duration = 15 # How long will our signal run for
fs = 1000 # What sampling frequency should we use
times = [2.5, 7.5, 12.5] # What times should the envelopes around the frequency specific signals be centred at?
widths = [12, 12, 12] # How wide should our pulses be (in time)?
frequencies = [3, 7, 11] # What frequencies should we embed in the signal?
amplitudes = [1, 1, 1] # What amplitude should the pulses be?

Using the variables above, create a `t` variable to align with our artificial signal (_hint: have a look at the gaussian pulse we made earlier_)

In [ ]:
# Generate a time vector
t = ...

Next, use list comprehention to create a list of pulses, iterating through the pulse times, widths, frequencies, and amplitudes (_hint: Yes, you can create a list of numpy arrays!_)

In [ ]:
# Loop through gaussian pulses that we want to generate
pulses = ...

You should now have a list of gaussian pulses, all with unique properties. However, in real data, these types of frequency and time specific pulses of activity would all be present within a single signal, and we would need to use computational methods to separate them out. To simulate this mixture of signals, let's add our lists together into a single signal (_hint: look at the documentation for [np.stack](https://numpy.org/doc/stable/reference/generated/numpy.stack.html) and [np.sum](https://numpy.org/doc/stable/reference/generated/numpy.sum.html)_)

In [ ]:
# Combine pulses into one signal. 
sig = ...

Congratulations, you have a signal! Use holoviews to visualise it, like we did to the gaussian pulses above.

In [ ]:
# Create a plot here!


## Step 6: Processing the artificial signal
We now have a single signal with a number of distinct frequency pulses imbedded within it. We neuroscientists work with signals like this all the time, and we have some tools to process them and segment out the activity at different frequencies and times. We'll run two signal processing steps in the next cell.

The first is a frequency transformation using [Welch's method](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.welch.html). This transformation allows us to visualise how strongly different frequencies are represented in our signal overall.  

The second is a time-frequency transformation using a Short Time Fourier Transform [STFT](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.stft.html). This transformation allows us to visualise how strongly different frequencies are represented in the signal at different times. 

Click the links to the documentation for the scipy functions implementing these two transforms to read more about them :)

In [ ]:
# Run Frequency transformation
f, Pxx = signal.welch(sig, fs, nperseg=fs*5)

# Run time-frequency transformation
f_spec, t_spec, spec = signal.spectrogram(sig, fs, nperseg=fs, noverlap=fs*3/4, scaling='density')

Now create a plot of the results of your frequency decomposition.

In [ ]:
power_dim = hv.Dimension('power', label='Power', unit='V^2/Hz')
time_dim = hv.Dimension('time', label='Time', unit='s')
freq_dim = hv.Dimension('freq', label='Frequency', unit='Hz')

plot_opts = (
    hv.opts.Curve(frame_width=100, frame_height=400, yaxis=None,
               ylim=(1, 50), line_color='yellow'),
    hv.opts.QuadMesh(frame_width=600, frame_height=400, ylim=(1, 50),
                    cmap='plasma')
)
(hv.QuadMesh((t_spec, f_spec, spec), kdims=[time_dim, freq_dim]) +
 hv.Curve((np.sqrt(Pxx), f), power_dim, freq_dim)).opts(*plot_opts)


## Step 7: Congratulations! 
You've completed phase 1 of the tutorial. See your friendly instructors Chris and Angie for further instruction :)